In [1]:
import numpy as np
import pandas as pd 

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

import glob
import cv2
import os


2.0.0


In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
sampleSabmission = pd.read_csv("sampleSubmission.csv")

In [3]:
train.head(3)


,Id,X,Y,Z,Time
0,1,2481900,528800,586,1.299872
1,2,2488400,530200,11686,1.679518
2,3,2495000,518400,6386,1.952326


In [4]:
test.head(3)

,Id,X,Y,Z
0,1,2488500,511100,11286
1,2,2471600,529600,2386
2,3,2483100,521300,10986


In [5]:
sampleSabmission.head(3)

,Id,Predicted
0,1,0
1,2,0
2,3,0


In [6]:
X_train_orig = train[["X", "Y", "Z"]]
X_test_orig  = test[["X", "Y", "Z"]]
Y_train_orig = train["Time"]

X_train_orig = np.asarray(X_train_orig)
Y_train_orig = np.asarray(Y_train_orig)
X_test_orig = np.asarray(X_test_orig)

print(X_train_orig.shape)
print(X_test_orig.shape)
print(Y_train_orig.shape)


(1086557, 3)
(1086558, 3)
(1086557,)


Let us now scale the data. We will use [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) from sklearn library.

The transformation is given by:

> X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) 

> X_scaled = X_std * (max - min) + min

In [7]:
scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
scaler.fit(X_train_orig)
print("Maximum values of X_train (X, Y, Z): " + str(scaler.data_max_))
print("Minimum values of X_train (X, Y, Z): " + str(scaler.data_min_))

# Use the same transform for both train and test data:
X_train_norm = scaler.transform(X_train_orig)
X_test_norm = scaler.transform(X_test_orig)

Maximum values of X_train (X, Y, Z): [2496500.  530500.   12986.]
Minimum values of X_train (X, Y, Z): [ 2.469e+06  5.010e+05 -2.140e+02]


In [8]:
# create training and testing vars

X_train, X_val, y_train, y_val = train_test_split(X_train_norm, Y_train_orig, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(869245, 3)
(869245,)
(217312, 3)
(217312,)


In [72]:
model = keras.Sequential([
    keras.layers.Dense(64, input_dim=3, activation='relu'),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

print(model.summary())

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_128 (Dense)            (None, 64)                256       
_________________________________________________________________
dense_129 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_130 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_131 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_132 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_133 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_134 (Dense)            (None, 8)               

In [73]:
model.compile(optimizer='adam',
              loss='MSE',
              metrics=['MAE'])

In [74]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=1024, epochs=20)

Train on 869245 samples, validate on 217312 samples
Epoch 1/20
869245/869245 [==============================] - 2s 2us/sample - loss: 0.1288 - MAE: 0.1270 - val_loss: 5.8147e-04 - val_MAE: 0.0180
Epoch 2/20
869245/869245 [==============================] - 1s 2us/sample - loss: 2.5850e-04 - MAE: 0.0121 - val_loss: 1.4971e-04 - val_MAE: 0.0096
Epoch 3/20
869245/869245 [==============================] - 1s 2us/sample - loss: 1.1975e-04 - MAE: 0.0085 - val_loss: 9.8768e-05 - val_MAE: 0.0077
Epoch 4/20
869245/869245 [==============================] - 1s 2us/sample - loss: 8.6888e-05 - MAE: 0.0073 - val_loss: 7.3302e-05 - val_MAE: 0.0067
Epoch 5/20
869245/869245 [==============================] - 2s 2us/sample - loss: 6.6387e-05 - MAE: 0.0064 - val_loss: 5.8297e-05 - val_MAE: 0.0060
Epoch 6/20
869245/869245 [==============================] - 1s 2us/sample - loss: 5.6550e-05 - MAE: 0.0059 - val_loss: 5.2550e-05 - val_MAE: 0.0057
Epoch 7/20
869245/869245 [==============================] - 1s 2

In [62]:
test_loss, test_acc = model.evaluate(X_val, y_val, verbose=0)
print('\nTest loss:', test_loss)
print('\nTest accuracy:', test_acc)


Test loss: 2.168219089727739e-05

Test accuracy: 0.0036353334


In [ ]:
print('\nhistory dict:', history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 4))
plt.subplot(2, 1, 1)
plt.plot(history.history['MAE'])
plt.plot(history.history['val_MAE'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.yscale('log')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.yscale('log')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# Generate predictions
predictions = model.predict(X_test_norm)

In [ ]:
mySubmission = sampleSabmission
mySubmission["Predicted"] = predictions

mySubmission.head()

filename = 'IvanPredictions_13.csv'

mySubmission.to_csv(filename,index=False)

print('Saved file: ' + filename)